<a href="https://colab.research.google.com/github/DhawaDong/Breast-Cancer-ANN/blob/main/LogisticRegression_BreastCancerDiagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd;
from sklearn.model_selection import train_test_split;
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [ ]:
DataSetBC = pd.read_csv(r'/content/drive/MyDrive/7#MachineLearning#Projects/BreastCancerDataSet.csv')

In [ ]:
DataSetBC.shape

In [ ]:
DataSetBC.head()

In [ ]:
x = DataSetBC.drop(['id', 'diagnosis', 'Unnamed: 32'], axis =1);   #pandas DataFrame
y = DataSetBC['diagnosis'];   # pandas series.

In [ ]:
DiagClass = {'M': 1.0, 'B': 0.0}
y = y.map(DiagClass);

In [ ]:
x.shape, y.shape

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
# Standardize features
scaler = StandardScaler()     #normalization for similar ranged mean and variance
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

#y_test_array = np.array(y_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size = 30):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear(x)
        out = self.sigmoid(out)
        return out

In [ ]:
# Instantiate the model
model = LogisticRegression()

In [ ]:
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  #SGD, Adagrad, RMSprop, Adadelta

In [ ]:
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.view(-1, 1))  #evaluate the loss value.

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()    #backpropagation, gradient of parameres(w) wrt error or loss.
    optimizer.step()   #adjust the new parameter values.

    # Print the loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Evaluate the model on the test set
with torch.no_grad():
    model.eval()    #set the model in evaluation mode
    predictions = model(X_test_tensor)
    predictions = (predictions >= 0.5).float()  # Convert probabilities to binary predictions
    accuracy = torch.sum(predictions == y_test_tensor.view(-1, 1)).item() / y_test_tensor.size(0)

In [ ]:
print(f'Test Accuracy: {accuracy:.4f}')